# Python and SQL cells in Snowflake Notebooks

One of the things I love about Snowflake Notebooks is how clean and easy it is to weave together Python and SQL. Sometimes it is easier to express certain operations in Python using DataFrames, and other times you might have some existing SQL script that I might want to reuse and build off of. With Snowflake Notebooks, you can do both — by using Python and SQL language cells to develop and run my code all in one place.

Let's take a look at a quick example below.

In [ ]:
# Create session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

Typically in a regular Python IDE, if you want to do anything SQL related, you would need to wrap the SQL query up in `session.sql` to run this SQL within my Python workflow. As you can see, this is not very clean from a code maintenance perspective — the stringified SQL is hard to read without code coloring, my notebook is now sprinkled with `session.sql` all over if I keep following this pattern.

In [ ]:
session.sql('''
CREATE OR REPLACE TABLE BOOK_CATALOG AS 
SELECT DATEADD(DAY, - UNIFORM (0, 180, RANDOM ()), CURRENT_DATE) AS TRANSACTION_DATE,
       CONCAT ('978',LPAD (ABS(UNIFORM (1000000000, 9999999999, RANDOM ())),10,'0')) AS ISBN, 
       ABS(10-NORMAL(5, 2, RANDOM())) AS RATING, 
       ABS(NORMAL(25, 30::FLOAT, RANDOM())) AS PRICE,
FROM TABLE(GENERATOR(ROWCOUNT => 100));
''')

In Snowflake Notebook, you can create SQL cells by clicking on the `+ SQL` button at the bottom of each cell, or change the language of your cell to `SQL` on the top left dropdown.

In [ ]:
-- SQL query for generating synthetic dataset
CREATE OR REPLACE TABLE BOOK_CATALOG AS 
SELECT DATEADD(DAY, - UNIFORM (0, 180, RANDOM ()), CURRENT_DATE) AS TRANSACTION_DATE,
       CONCAT ('978',LPAD (ABS(UNIFORM (1000000000, 9999999999, RANDOM ())),10,'0')) AS ISBN, 
       ABS(10-NORMAL(5, 2, RANDOM())) AS RATING, 
       ABS(NORMAL(25, 30::FLOAT, RANDOM())) AS PRICE,
FROM TABLE(GENERATOR(ROWCOUNT => 100));

Now I can use Python to access this table as a dataframe and print it out.

In [ ]:
# Start working with my Snowflake table in Python
df = session.table("BOOK_CATALOG")
df